In [1]:
import geopandas as gpd
import pandas as pd
import warnings
from tqdm.contrib.concurrent import process_map
warnings.filterwarnings('ignore')
from multiprocessing import Process, Manager

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
kdprov='51'

In [3]:
data_rai=gpd.read_file('data/input/RBI/kdprov_'+kdprov+'.gpkg').to_crs('ESRI:54034').dissolve()

In [4]:
nmfiledegurba="degurba_Bali_00.gpkg"
gf=gpd.read_file("data/output/2_grid_degurba/"+nmfiledegurba).to_crs('ESRI:54034')
gf['RAI']=0

In [5]:
data_desa=gpd.read_file('data/input/Master_Data/DESA_20201_BYINDO.gpkg').to_crs('ESRI:54034').query('kdprov==@kdprov')
gf=gf.sjoin(data_desa[['geometry']])[gf.columns]

In [6]:
def data_masking_degurba(i):
    global degurba_desa,data_highway,gf,affected_RAI
    degurba_desa=gpd.sjoin(gf,data_desa.query('iddesa==@i'))
    data_highway_j=data_rai.overlay(degurba_desa,how='intersection')[['geometry']]
    data_highway_j['reg']=1
    data_highway_j=data_highway_j.dissolve(by='reg')
    data_highway_j.geometry=data_highway_j.buffer(2000)
    nm_rai=gf.sjoin(data_highway_j)['Name'].tolist()
    affected_RAI.extend(nm_rai)
    
    #print(affected_RAI)

In [7]:
for j in data_desa.idkab.unique():
    print('Running for ',j)
    data_desa_kab=data_desa.query('idkab==@j')
    manager=Manager()
    affected_RAI=manager.list()
    process_map(data_masking_degurba,data_desa_kab.iddesa.unique())
    gf[gf.Name.isin(list(affected_RAI))].to_file('data/output/4_RAI/rai_rbi_degurba_'+j+'.gpkg',driver='GPKG')

Running for  5101


  0%|          | 0/51 [00:00<?, ?it/s]

Running for  5102


  0%|          | 0/133 [00:00<?, ?it/s]

Running for  5103


...

Running for  5108


  0%|          | 0/148 [00:00<?, ?it/s]

Running for  5171


  0%|          | 0/43 [00:00<?, ?it/s]

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
from glob import glob
degurba_rai=glob('data/output/4_RAI/rai_rbi_degurba_'+kdprov+'*.gpkg')

In [ ]:
data_=gpd.GeoDataFrame()
l_name=[]
for i in degurba_rai:
    temp=gpd.read_file(i)['Name']
    l_name.extend(list(temp))
l_name=list(set(l_name))

In [ ]:
gf['accessed']=gf.Name.apply(lambda y: 1 if y in l_name else 0)

In [ ]:
gf.to_file('data/output/4_RAI/fin_rai_rbi_degurba_'+kdprov+'.gpkg')

In [ ]:
data_p=gf.pivot_table(index='classification',columns='accessed',values='pop',aggfunc='sum').fillna(0)

In [ ]:
data_p['percent']=data_p[1]*100/(data_p[0]+data_p[1])

In [ ]:
data_p